# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [28]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [29]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [30]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis = 1)
new_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [31]:
!pip install surprise

In [32]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 

reader = Reader()
data = Dataset.load_from_df(new_df, reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [33]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [34]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [35]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
param_grid = {'n_factors': [20, 10], 
              'n_epochs': [5, 10], 
              'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}

grid = GridSearchCV(SVD, param_grid = param_grid, n_jobs = -1, cv = 3, joblib_verbose = 5)
grid.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.8min finished


In [36]:
# print out optimal parameters for SVD after GridSearch

print(grid.best_score)
print(grid.best_params)

{'rmse': 0.8941374128234271, 'mae': 0.6921658063212625}
{'rmse': {'n_factors': 20, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}, 'mae': {'n_factors': 10, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}}


In [37]:
# cross validating with KNNBasic

knn_basic = KNNBasic(sim_options = {'name':'pearson', 'user_based': True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs = -1)

In [38]:
# print out the average RMSE score for the test set

for i in cv_knn_basic.items():
    print(i)
    
print('------------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.98613661, 0.9679074 , 0.97016177, 0.97727885, 0.96173963]))
('test_mae', array([0.76157713, 0.74662698, 0.74970509, 0.75429109, 0.74423771]))
('fit_time', (0.6037511825561523, 0.6308736801147461, 0.6599700450897217, 0.6652348041534424, 0.5966808795928955))
('test_time', (1.8732879161834717, 1.933568000793457, 1.9951062202453613, 1.8787477016448975, 1.7589662075042725))
------------------------
0.972644851039908


In [39]:
# cross validating with KNNBaseline

knn_baseline = KNNBaseline(sim_options = {'name':'pearson', 'user_based': True})
cv_knn_baseline = cross_validate(knn_baseline, data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [40]:
# print out the average score for the test set

for i in cv_knn_baseline.items():
    print(i)
    
np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0.88577014, 0.88932028, 0.86855133, 0.87254823, 0.87470104]))
('test_mae', array([0.67947294, 0.67783925, 0.66601221, 0.66467048, 0.66771542]))
('fit_time', (0.8791801929473877, 0.8731689453125, 0.8801312446594238, 0.8753700256347656, 0.8902771472930908))
('test_time', (2.6166059970855713, 2.6590399742126465, 2.6501359939575195, 2.678581953048706, 2.5724849700927734))


0.8781782041522692

Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [41]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [42]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [43]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [44]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.145557141000333, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [53]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list    

In [54]:
# try out the new function here!

user_rating = movie_rater(df_movies, 4, 'Comedy')

      movieId                    title                        genres
4691     7004  Kindergarten Cop (1990)  Action|Comedy|Crime|Thriller
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
     movieId         title                       genres
520      608  Fargo (1996)  Comedy|Crime|Drama|Thriller
How do you rate this movie on a scale of 1-5, press n if you have not seen :
5
      movieId                        title  genres
3131     4214  Revenge of the Nerds (1984)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3
      movieId          title                    genres
1551     2088  Popeye (1980)  Adventure|Comedy|Musical
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [55]:
user_rating

[{'userId': 1000, 'movieId': 7004, 'rating': '4'},
 {'userId': 1000, 'movieId': 608, 'rating': '5'},
 {'userId': 1000, 'movieId': 4214, 'rating': '3'},
 {'userId': 1000, 'movieId': 2088, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [56]:
## add the new ratings to the original ratings DataFrame

new_ratings_df = new_df.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [57]:
new_ratings_df.head()

,userId,movieId,rating
0,1,1,4
1,1,3,4
2,1,6,4
3,1,47,5
4,1,50,5


In [59]:
# train a model using the new combined DataFrame

svd_ = SVD(n_factors = 50, reg_all = 0.05)
svd_.fit(new_data.build_full_trainset())

In [60]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)

list_of_movies = []
for m_id in new_df['movieId'].unique():
        list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))

In [61]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(list_of_movies, key = lambda x: x[1], reverse = True)

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [62]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx + 1, ': ', title, '\n')
            n -= 1
            if n == 0:
                break
                
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Recommendation #  2 :  899    Princess Bride, The (1987)
Name: title, dtype: object 

Recommendation #  3 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  4 :  900    Raiders of the Lost Ark (Indiana Jones and the...
Name: title, dtype: object 

Recommendation #  5 :  46    Usual Suspects, The (1995)
Name: title, dtype: object 



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.